<a href="https://colab.research.google.com/github/jeffvun/Machine-Learning-Labs/blob/main/Image_Forensics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Libraries**

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import FasterRCNN, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model


## **Data Preprocessing**

In [ ]:
# Define your data directories
data_dir = '/Content/data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# Image data generator for data augmentation
datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)

# Create generators for training and testing data
batch_size = 32
train_generator = datagen.flow_from_directory(train_dir,
                                              target_size=(224, 224),
                                              batch_size=batch_size,
                                              class_mode='binary')

test_generator = datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=batch_size,
                                             class_mode='binary')


## **Product Identification using R-CNN**

In [ ]:
# Use a pre-trained ResNet50 as the base model for Faster R-CNN
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)

model_frcnn = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_frcnn.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

# Train the model
model_frcnn.fit(train_generator, epochs=5, validation_data=test_generator)
